In [22]:
from pathlib import Path
from dl_toolbox.datasets import xView1
from pycocotools.coco import COCO
from dl_toolbox.utils import get_tiles
from collections import defaultdict

In [26]:
LABEL_TO_STRING = {
    11: "Fixed-wing Aircraft",
    12: "Small Aircraft",
    13: "Passenger/Cargo Plane",
    15: "Helicopter",
    17: "Passenger Vehicle",
    18: "Small Car",
    19: "Bus",
    20: "Pickup Truck",
    21: "Utility Truck",
    23: "Truck",
    24: "Cargo Truck",
    25: "Truck Tractor w/ Box Trailer",
    26: "Truck Tractor",
    27: "Trailer",
    28: "Truck Tractor w/ Flatbed Trailer",
    29: "Truck Tractor w/ Liquid Tank",
    32: "Crane Truck",
    33: "Railway Vehicle",
    34: "Passenger Car",
    35: "Cargo/Container Car",
    36: "Flat Car",
    37: "Tank car",
    38: "Locomotive",
    40: "Maritime Vessel",
    41: "Motorboat",
    42: "Sailboat",
    44: "Tugboat",
    45: "Barge",
    47: "Fishing Vessel",
    49: "Ferry",
    50: "Yacht",
    51: "Container Ship",
    52: "Oil Tanker",
    53: "Engineering Vehicle",
    54: "Tower crane",
    55: "Container Crane",
    56: "Reach Stacker",
    57: "Straddle Carrier",
    59: "Mobile Crane",
    60: "Dump Truck",
    61: "Haul Truck",
    62: "Scraper/Tractor",
    63: "Front loader/Bulldozer",
    64: "Excavator",
    65: "Cement Mixer",
    66: "Ground Grader",
    71: "Hut/Tent",
    72: "Shed",
    73: "Building",
    74: "Aircraft Hangar",
    76: "Damaged Building",
    77: "Facility",
    79: "Construction Site",
    83: "Vehicle Lot",
    84: "Helipad",
    86: "Storage Tank",
    89: "Shipping container lot",
    91: "Shipping Container",
    93: "Pylon",
    94: "Tower",
}
STRING_TO_LABEL = {v: k for k, v in LABEL_TO_STRING.items()}

In [23]:
coco = COCO(Path('/data')/'XVIEW1'/'xView_train.json')

loading annotations into memory...
Done (t=4.05s)
creating index...
index created!


In [31]:
import pandas as pd

df = pd.DataFrame(columns=['image_id', 'left', 'top', 'width', 'height'])
for name in list(STRING_TO_LABEL.keys()):
    df[name] = 0

In [86]:
data = []
for i, img_id in enumerate(coco.imgs.keys()):
    img = coco.imgs[img_id]
    W, H = img['width'], img['height']
    tiles = get_tiles(
        W,
        H,
        width=672,
        step_w=336
    )        
    for l, t, w, h in tiles:
        row = {'image_id': img_id, 'left': l, 'top': t, 'width': w, 'height': h}
        counts = defaultdict(int)
        for ann in coco.imgToAnns[img_id]:
            cat_id = ann['category_id']
            bbx,bby,bbw,bbh = ann['bbox']
            cx, cy = bbx+bbw/2, bby+bbh/2
            if l < cx < l+w and t < cy < t+h and cat_id in coco.cats.keys():
                cat_name = LABEL_TO_STRING[cat_id]
                counts[cat_name] += 1
        row.update(counts)
        data.append(row.copy())
        #break
    if i%100==0:
        print(i)

df = pd.DataFrame(data, columns=['image_id', 'left', 'top', 'width', 'height']+list(STRING_TO_LABEL.keys())).fillna(0).astype('int64')
df.to_csv(Path('/data')/'XVIEW1'/'df.csv')

0
100
200
300
400
500
600
700
800


In [ ]:
data_path = Path('/data')
coco = COCO(data_path/'XVIEW1'/'xView_train.json')
df = pd.read_csv(data_path/'XVIEW1'/'df.csv', index_col=0)
filtered = df.loc[df['Building'] != 0.][['image_id', 'left', 'top', 'width', 'height', 'Building']] # A changer pour merge
ids_tiles = []
num_obj_per_tile = []
for _, [img_id, l, t, w, h, num] in filtered.iterrows():
    print(img_id)
    ids_tiles.append((img_id, (l,t,w,h)))
    num_obj_per_tile.append(num)
nb_imgs = len(ids_tiles)
val_start, test_start = int(0.8*nb_imgs), int(0.85*nb_imgs)
train_ids, val_ids = ids_tiles[:val_start], ids_tiles[val_start:test_start]
train_ids_and_windows = ids_tiles[:val_start]
val_ids_and_windows = ids_tiles[val_start:test_start]
    
train_set = xView1(
    merge='building',
    coco_dataset=coco,
    root=data_path/'XVIEW1'/'train_images',
    ids=train_ids_and_windows
)

In [85]:
train_set[0]

{'image': Image([[[0.0275, 0.0235, 0.0235,  ..., 0.1255, 0.1294, 0.1294],
         [0.0275, 0.0314, 0.0235,  ..., 0.1255, 0.1333, 0.1333],
         [0.0235, 0.0275, 0.0235,  ..., 0.1373, 0.1373, 0.1333],
         ...,
         [0.0314, 0.0275, 0.0275,  ..., 0.5255, 0.5294, 0.5373],
         [0.0275, 0.0314, 0.0353,  ..., 0.5333, 0.5412, 0.5333],
         [0.0235, 0.0235, 0.0353,  ..., 0.5412, 0.5451, 0.5294]],
 
        [[0.0275, 0.0275, 0.0235,  ..., 0.1098, 0.1137, 0.1137],
         [0.0314, 0.0353, 0.0275,  ..., 0.1098, 0.1176, 0.1176],
         [0.0275, 0.0275, 0.0275,  ..., 0.1176, 0.1176, 0.1137],
         ...,
         [0.0392, 0.0392, 0.0392,  ..., 0.5216, 0.5216, 0.5294],
         [0.0353, 0.0392, 0.0471,  ..., 0.5255, 0.5333, 0.5255],
         [0.0314, 0.0275, 0.0431,  ..., 0.5294, 0.5333, 0.5216]],
 
        [[0.0118, 0.0118, 0.0118,  ..., 0.0549, 0.0588, 0.0588],
         [0.0118, 0.0118, 0.0078,  ..., 0.0549, 0.0588, 0.0588],
         [0.0118, 0.0118, 0.0078,  ..., 0.0627,

In [1]:





class COCO_xview(COCO):
    
    def __init__(self):
        super().__init__()
        
    def createIndex(self):
        # create index
        print('creating index...')
        anns, cats, imgs = {}, {}, {}
        imgToAnns,catToImgs = defaultdict(list),defaultdict(list)

        if 'images' in self.dataset:
            for img in self.dataset['images']:
                imgs[img['id']] = img

        if 'categories' in self.dataset:
            for cat in self.dataset['categories']:
                cats[cat['id']] = cat
                
        if 'annotations' in self.dataset:
            for ann in self.dataset['annotations']:
                anns[ann['id']] = ann
                img_id = ann['image_id']
                x,y,w,h = ann['bbox']
                imgToAnns[img_id].append(ann)

        if 'annotations' in self.dataset and 'categories' in self.dataset:
            for ann in self.dataset['annotations']:
                catToImgs[ann['category_id']].append(ann['image_id'])

        print('index created!')

        # create class members
        self.anns = anns
        self.imgToAnns = imgToAnns
        self.catToImgs = catToImgs
        self.imgs = imgs
        self.cats = cats
    
    def getImgIdsAndTiles(catIds, tile_size=1000):
        imgIds = []
        catIds = catIds if _isArrayLike(catIds) else [catIds]
        if len(catIds) == 0:
            ids = self.imgs.keys()
        else:
            catToImgsAndTiles = defaultdict(list)
            for ann in self.dataset['annotations']:
                img_id = ann['image_id']
                x,y,w,h = ann['bbox']
                cx, cy = x+w/2, y+h/2
                tile = int(center/tile_size
                catToImgsAndTiles[ann['category_id']].append
                
                
                
            for i, catId in enumerate(catIds):
                cat_ids = set(self.catToImgs[catId])
                for id in cat_ids:
                    img = self.imgs[id]
                    for tile in get_tiles(
                        img['width'],
                        img['height'],
                        width=tile_size,
                        step_w=tile_step
                    ):
                        
                if i == 0 and len(ids) == 0:
                    
                else:
                    ids &= set(self.catToImgs[catId])
        return list(ids)

coco = COCO(Path('/data')/'XVIEW1'/'xView_train.json')
ids = [id for id, tile in coco.getImgIds(catIds=73) if id in coco.imgs.keys()]
ids_and_windows = []
for id in ids:
    img = coco.imgs[id]
    tiles = get_tiles(
        img['width'],
        img['height'],
        width=img['width'],
        height=img['height'],
        step_w=256
    )
    ids_and_windows += [(id, w) for w in tiles]
    
dataset = xView1(
    merge='building',
    coco_dataset=coco,
    root=Path('/data')/'XVIEW1'/'train_images',
    ids=ids_and_windows
)
print(len(dataset))

loading annotations into memory...
Done (t=3.28s)
creating index...
index created!
726


In [21]:
list(get_tiles(1000,1000,200))

[(0, 0, 200, 200),
 (0, 200, 200, 200),
 (0, 400, 200, 200),
 (0, 600, 200, 200),
 (0, 800, 200, 200),
 (200, 0, 200, 200),
 (200, 200, 200, 200),
 (200, 400, 200, 200),
 (200, 600, 200, 200),
 (200, 800, 200, 200),
 (400, 0, 200, 200),
 (400, 200, 200, 200),
 (400, 400, 200, 200),
 (400, 600, 200, 200),
 (400, 800, 200, 200),
 (600, 0, 200, 200),
 (600, 200, 200, 200),
 (600, 400, 200, 200),
 (600, 600, 200, 200),
 (600, 800, 200, 200),
 (800, 0, 200, 200),
 (800, 200, 200, 200),
 (800, 400, 200, 200),
 (800, 600, 200, 200),
 (800, 800, 200, 200)]

In [5]:
import torch
import pandas as pd

def num_boxes(dataset):
    res = []
    for i, x in enumerate(dataset):
        #if i%10==0: print(i)
        res.append(int(x['target']['labels'].shape[0]))
    return res
    
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices: a list of indices
        num_samples: number of samples to draw
        callback_get_label: a callback-like function which takes two arguments - dataset and index
    """

    def __init__(
        self,
        dataset,
        labels = None,
        num_samples = None,
        callback = None,
    ):
        # if indices is not provided, all elements in the dataset will be considered
        self.indices = list(range(len(dataset)))
        
        # if num_samples is not provided, draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) if num_samples is None else num_samples

        # distribution of classes in the dataset
        df = pd.DataFrame()
        df["num_boxes"] = callback(dataset)
        df.index = self.indices
        df = df.sort_index()

        weights = df["num_boxes"]

        self.weights = torch.DoubleTensor(weights.to_list())

    def __iter__(self):
        return (
            self.indices[i]
            for i in torch.multinomial(self.weights, self.num_samples, replacement=True)
        )

    def __len__(self):
        return self.num_samples

In [6]:
rnd_sampler=torch.utils.data.RandomSampler(
    dataset,
    replacement=True,
    num_samples=100
)

In [7]:
weighted_sampler = ImbalancedDatasetSampler(
    dataset,
    callback=num_boxes,
    num_samples=100
)

In [8]:
rnd_num_boxes = []
for indice in rnd_sampler:
    rnd_num_boxes.append(int(dataset[indice]['target']['labels'].shape[0]))

In [9]:
print(rnd_num_boxes)

[55, 31, 536, 3, 809, 8, 240, 35, 3, 13, 226, 756, 147, 2, 48, 507, 67, 35, 16, 1361, 506, 1137, 23, 10, 2, 793, 1257, 466, 885, 38, 6, 346, 383, 883, 24, 1, 340, 1, 286, 3269, 958, 55, 52, 706, 16, 140, 7, 76, 15, 983, 31, 2354, 43, 2441, 13, 1196, 534, 289, 530, 48, 51, 73, 204, 1044, 3, 700, 128, 915, 240, 50, 104, 3, 1120, 1, 1351, 6, 199, 8, 1946, 231, 1373, 2007, 6, 7, 517, 83, 28, 8, 452, 4, 122, 20, 2, 7, 14, 121, 17, 40, 5063, 3269]


In [10]:
weighted_num_boxes = []
for indice in weighted_sampler:
    weighted_num_boxes.append(int(dataset[indice]['target']['labels'].shape[0]))

In [11]:
print(weighted_num_boxes)

[5063, 3269, 1345, 1440, 3032, 3613, 2339, 5063, 375, 2299, 1351, 3311, 3398, 756, 4924, 2331, 1221, 1642, 3847, 254, 3311, 5, 864, 1124, 3311, 1515, 1914, 3221, 1635, 2354, 484, 993, 279, 161, 1377, 3068, 218, 2044, 216, 2260, 860, 226, 1589, 542, 1114, 1014, 216, 1082, 1423, 1347, 3347, 41, 1951, 3032, 2563, 1946, 734, 655, 944, 5063, 3398, 81, 522, 3381, 3381, 1373, 43, 3032, 166, 2483, 1157, 2478, 2715, 1347, 3311, 3268, 3398, 507, 1563, 1698, 864, 853, 289, 7429, 3068, 220, 1563, 4924, 7, 756, 998, 92, 2044, 1345, 2483, 530, 885, 1421, 7429, 3847]


In [15]:
img_paths = [Path('/data')/'XVIEW1'/'train_images'/coco.loadImgs(id)[0]["file_name"] for id in coco.imgs.keys()]

In [17]:
coco.loadImgs(774)

[{'id': 774,
  'width': 2773,
  'height': 2678,
  'file_name': '774.tif',
  'license': 1}]

In [16]:
import rasterio
import rasterio.windows as win

class xViewCOCO

coco = COCO(self.data_path/'XVIEW1'/'xView_train.json')
ids = []
merges = [list(l.values) for l in self.class_list]
for merge in merges:
    # It seems that xview annotations geojson contain bboxes for images not in train_images nor val (test set not avail?)
    ids += [id for id in coco.getImgIds(catIds=merge) if id in coco.imgs.keys()]
nb_imgs = len(ids)
val_start, test_start = int(0.8*nb_imgs), int(0.85*nb_imgs)
train_ids, val_ids = ids[:val_start], ids[val_start:test_start]
self.train_ids_and_windows = []
for id in train_ids:
    img = coco.imgs[id]
    tiles = get_tiles(
        img['width'],
        img['height'],
        self.tile_size,
        step_w=256
    )
    self.train_ids_and_windows += [(id, w) for w in tiles]

df = pd.DataFrame()

for id in coco.imgs.keys()[:2]:
    img_desc = coco.loadImgs(id)[0]
    path = Path('/data')/'XVIEW1'/'train_images'/img_desc["file_name"]
    width = img_desc["width"]
    height = img_desc["height"]
    tiles = get_tiles(
        width,
        height,
        width=1000,
        step_w=900
    )
    windows = [win.Window(l,t,w,h) for l,t,w,h in tiles]
    
    with rasterio.open(path, "r") as file:
        image = file.read(out_dtype=np.uint8)
        
    ann_ids = coco.getAnnIds(id)
    for ann_id in ann_ids:
        ann = dict(self.coco.anns[ann_id])
        bb_win = win.Window(*ann['bbox'])
        if win.intersect([window, bb_win]):
            x,y,w,h = win.intersection([window, bb_win]).flatten()
            x -= l
            y -= t
            ann['bbox'] = (x,y,w,h)
            anns.append(ann)

[PosixPath('/data/XVIEW1/train_images/774.tif'),
 PosixPath('/data/XVIEW1/train_images/1786.tif'),
 PosixPath('/data/XVIEW1/train_images/2486.tif'),
 PosixPath('/data/XVIEW1/train_images/1509.tif'),
 PosixPath('/data/XVIEW1/train_images/1090.tif'),
 PosixPath('/data/XVIEW1/train_images/181.tif'),
 PosixPath('/data/XVIEW1/train_images/223.tif'),
 PosixPath('/data/XVIEW1/train_images/2505.tif'),
 PosixPath('/data/XVIEW1/train_images/510.tif'),
 PosixPath('/data/XVIEW1/train_images/1046.tif')]

In [ ]:
df = pd.DataFrame()
df["num_boxes"] = callback(dataset)
df.index = self.indices
df = df.sort_index()